In [ ]:
import os
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import mobilenet_v3_small, MobileNet_V3_Small_Weights
import torch.nn as nn
import torch.optim as optim

from PIL import Image
import pandas as pd
from tqdm import tqdm

# Configuration
DATA_DIR = os.path.expanduser('~/GroceryStoreDataset/dataset')
BATCH_SIZE = 64
NUM_EPOCHS = 10
NUM_CLASSES = 81  # Fine-grained classes
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

ModuleNotFoundError: No module named 'torch'

In [ ]:
class GroceryDataset(Dataset):
    def __init__(self, txt_file, transform=None):
        self.samples = []
        with open(txt_file, 'r') as f:
            for line in f:
                parts = line.strip().split(",")
                if len(parts) >= 2:
                    img_path = os.path.join(DATA_DIR, parts[0])
                    label = int(parts[1])  # Fine-grained label
                    self.samples.append((img_path, label))
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

# Define transforms
transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Create datasets
train_dataset = GroceryDataset(os.path.join(DATA_DIR, 'train.txt'), transform=transform)
val_dataset = GroceryDataset(os.path.join(DATA_DIR, 'val.txt'), transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [ ]:
# Load pre-trained MobileNetV3 Small model
model = mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.DEFAULT)

# Replace the classifier to match the number of classes
model.classifier[3] = nn.Linear(model.classifier[3].in_features, NUM_CLASSES)

model = model.to(DEVICE)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.9)

for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}"):
        images, labels = images.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Training Loss: {epoch_loss:.4f}")
    # scheduler.step()


In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Validation Accuracy: {accuracy:.2%}')


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import torch
import random
from torchvision import transforms
from PIL import Image
from pathlib import Path

# Load class index to name mapping
classes_csv_path = Path('~/GroceryStoreDataset/dataset/classes.csv').expanduser()
classes_df = pd.read_csv(classes_csv_path, header=None, names=['class_id', 'fine_label', 'coarse_label', 'iconic_image', 'description'])
class_id_to_name = dict(zip(classes_df['class_id'], classes_df['fine_label']))

# Define the number of samples to display
num_samples = 8

# Set the model to evaluation mode
model.eval()

# Select random samples from the validation dataset
indices = random.sample(range(len(val_dataset)), num_samples)

# Create a figure to display the images
fig, axes = plt.subplots(1, num_samples, figsize=(20, 5))

for idx, ax in zip(indices, axes):
    # Get the image and label
    image, true_label = val_dataset[idx]
    image_tensor = image.unsqueeze(0).to(device)

    # Get the model prediction
    with torch.no_grad():
        output = model(image_tensor)
        _, predicted_label = torch.max(output, 1)

    # Convert the image tensor to a PIL image for display
    image_display = transforms.ToPILImage()(image.cpu())

    # Display the image
    ax.imshow(image_display)
    ax.axis('off')

    # Set the title with predicted and true labels
    pred_name = class_id_to_name.get(predicted_label.item(), 'Unknown')
    true_name = class_id_to_name.get(true_label, 'Unknown')
    ax.set_title(f'Pred: {pred_name}\nTrue: {true_name}', fontsize=10)

plt.tight_layout()
plt.show()
